In [1]:
#ライブラリ
import tweepy
import nltk
import stanza
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
#Twitter API config読み込み
config = pd.read_csv("./config.csv")

In [3]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [4]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [25]:
# キーワードからツイートを取得
s='btc min_faves:1 min_retweets:1 -filter:retweets'
tweets = tweepy.Cursor(api.search,  
                        count=10,
                        q=s,
                        exclude_replies=True,
                        tweet_mode='extended',
                        show_user=True,
                        lang = 'en',
                        since='2021-07-14',
                        until='2021-07-15',
                        ).items(1000);

In [26]:
#ツイートのリスト
t_data = []
for tweet in tweets:
    t_data.append([tweet.user.name,'@'+tweet.user.screen_name,tweet.user.friends_count,tweet.user.followers_count,tweet.full_text.replace('\n',''),tweet.favorite_count,tweet.retweet_count,tweet.created_at])

In [27]:
#処理前ツイート出力
df = pd.DataFrame(data=t_data, columns=['Name','User Name','Following','Followers','Tweet','Favorite','RT','created at'])

In [28]:
#処理前ツイート先頭10件
df.head(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,created at
0,MINISHIBA SHILLER,@Punderdeep,44,5,@cz_binance 🪙 Mini Shiba Inu 🪙THE NEXT 1000x G...,9,9,2021-07-14 23:59:58
1,MINISHIBA SHILLER,@Punderdeep,44,5,@cz_binance 🪙 Mini Shiba Inu 🪙THE NEXT 1000x G...,7,7,2021-07-14 23:59:07
2,Pi Prime Pi,@PiPrimePi,267,2469,Spike Lee clearly isn’t paying attention to wh...,14,1,2021-07-14 23:59:06
3,Cryptonite⚡,@Teaching_Crypto,901,32972,Depends on $Btc tho. https://t.co/rgmKMVSb7N,3,1,2021-07-14 23:58:35
4,Vinay,@vinay_93,238,38,People will always complain but what will you ...,2,1,2021-07-14 23:58:16
5,MINISHIBA SHILLER,@Punderdeep,44,5,@bsc_daily @WSGToken 🪙 Mini Shiba Inu 🪙THE NEX...,6,6,2021-07-14 23:57:46
6,CryptoBlockParty,@CryptoBlckParty,4996,884,#ETH True #Layer2 #Uniswapv3 @Uniswap #Layer#O...,4,2,2021-07-14 23:57:40
7,𝚃𝚒𝚏𝚏𝚊𝚗𝚢 𝙷𝚊𝚢𝚍𝚎𝚗,@haydentiff,6375,75007,@sanssomatic @EsportsDealer @noobudee @Bennett...,1,2,2021-07-14 23:56:53
8,CryptoBlockParty,@CryptoBlckParty,4996,884,#ETH #Layer2 #Uniswapv3 @Uniswap #Layer#Optimi...,5,1,2021-07-14 23:56:41
9,Dick Whitman Capital,@DickWhitman_BTC,1654,2275,@ShortNon @stacksaurus @Leishman CBDC’s: “I go...,4,1,2021-07-14 23:56:03


In [29]:
#処理前ツイート末尾10件
df.tail(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT,created at
990,Captain BTC 🅡🥛,@Captain_BTC,948,4999,"Staying bullish, this cycle isnt over",12,3,2021-07-14 17:59:26
991,Bitcoin Archive 🗄🚀🌔,@BTC_Archive,584,380138,Inflation in action at @Target. @HP laptop goe...,595,96,2021-07-14 17:59:05
992,TheLatestBlock,@TheLatestBlock,574,387,#crypto news in 1 Minute with @alexandre_lores...,3,2,2021-07-14 17:59:02
993,Alberto David Mercado,@AlbertoDMercado,812,777,#bitcoin looking green today. Hopefully. $ZIL ...,5,1,2021-07-14 17:58:55
994,🇺🇲🇮🇹3ℓℓ3🇵🇹🇪🇺,@B055Lady_Elle,12336,12185,🐩POODL FLASH SALE LIVE thru Friday - 100% TAX ...,1,5,2021-07-14 17:58:32
995,Bitcoinmaster1983,@Bitcoin22109717,109,32,Dont miss the oportunity to win 1 billion #poo...,3,4,2021-07-14 17:57:56
996,Russian Bitcoin 🇷🇺,@Russian_BTC,130,1292,@safemoon @binance @papacthulu Take Off now 3 ...,29,5,2021-07-14 17:57:44
997,SmileyGnome BTC 🏧,@SmileyGnome,467,4627,More components arrived to perfect our prototy...,27,13,2021-07-14 17:57:36
998,Nolan Koen,@nolankoen,87,119,#Bitcoin moving in this downward sloping patte...,1,1,2021-07-14 17:57:25
999,Nossie 🍀 🇳🇱,@Nossie_NL,13,43,https://t.co/d1nxYwV3DoLook at this video and ...,2,2,2021-07-14 17:57:00


In [30]:
#処理前出力
df.to_csv("main.csv")

In [ ]:
#ツイート前処理
def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt)
    return txt

In [ ]:
#lemmatization
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def Lemma(txt):
    doc = nlp(txt)
    for sent in doc.sentences:
        for word in sent.words:
            return word.lemma, end = " "
        
df['Score'] = df['Tweet'].apply(getTextPolarity)

In [ ]:
#VADER感情分析器インポート
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
#感情出力関数
def getTextPolarity(txt):
    return vader_analyzer.polarity_scores(txt)
def getTextPos(txt):
    return txt.get("pos")
def getTextNeu(txt):
    return txt.get("neu")
def getTextNeg(txt):
    return txt.get("neg")
def getTextCom(txt):
    return txt.get("compound")

def getTextAnalysis(l):
    s = l.get("compound")
    if s < 0:
        return "Negative"
    elif s == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
#表に分析結果を追加
df['Score'] = df['Tweet'].apply(getTextPolarity)
df['neg'] = df['Score'].apply(getTextNeg)
df['neu'] = df['Score'].apply(getTextNeu)
df['pos'] = df['Score'].apply(getTextPos)
df['compound'] = df['Score'].apply(getTextCom)
df['Sentiment'] = df['Score'].apply(getTextAnalysis)

In [ ]:
#感情数値表示
df.drop(columns='Score')

In [ ]:
#結果を表に上書き
df.to_csv("main.csv")